# Data Loading

In [ ]:
import pandas as pd

df = pd.ExcelFile(r'C:/Users/hassa/TimeTable, FSC, Fall-2022.xlsx')

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
timetable = {day: pd.read_excel(df, sheet_name=day, skiprows=4).set_index('Room').dropna(how='all', axis=1) for day in days}

# Functions To Be Used

In [6]:
def getClassesForSubject(subject):
    result = []
    for day, dayDf in timetable.items():
        for room in dayDf.index:
            for timeSlot, value in dayDf.loc[room].items():
                if pd.notna(value) and subject.lower() in value.lower():
                    result.append((day, timeSlot, room, value))
    return result

def getClassesForSubjects(subjects):
    result = []
    for subject in subjects:
        result.extend(getClassesForSubject(subject))
    return result

def getEmptySlots(day):
    dayDf = timetable[day]
    emptySlots = dayDf.applymap(lambda x: pd.isna(x)).all(axis=1)
    return emptySlots

def getLeastBusyClassroom():
    roomUsage = {}
    for day, dayDf in timetable.items():
        for room in dayDf.index:
            if room not in roomUsage:
                roomUsage[room] = 0
            roomUsage[room] += dayDf.loc[room].notna().sum().sum()
    return min(roomUsage, key=roomUsage.get)

# Get Classes For Single Subject

In [7]:
subject = input("Enter the subject: ")
classes = getClassesForSubject(subject)
for cls in classes:
    print(f"Day: {cls[0]}, Time: {cls[1]}, Room: {cls[2]}, Class: {cls[3]}")

Enter the subject: PF (CS-A)
Day: Monday, Time: 08:30-09:50, Room: C-301, Class: PF (CS-A)
Day: Wednesday, Time: 08:30-09:50, Room: C-301, Class: PF (CS-A)


# Get Classes For Multiple Subjects

In [8]:
subjects = input("Enter the subjects (comma separated): ").split(',')
classes = getClassesForSubjects(subjects)
for cls in classes:
    print(f"Day: {cls[0]}, Time: {cls[1]}, Room: {cls[2]}, Class: {cls[3]}")

Enter the subjects (comma separated): Calculus (CS-A),AP (CS-A)
Day: Monday, Time: 10:00-11:20, Room: C-301, Class: Calculus (CS-A)
Day: Wednesday, Time: 10:00-11:20, Room: C-301, Class: Calculus (CS-A)
Day: Monday, Time: 11:30-12:50, Room: C-301, Class: AP (CS-A)
Day: Wednesday, Time: 11:30-12:50, Room: C-301, Class: AP (CS-A)
Day: Thursday, Time: 11:30-12:50, Room: B-130, Class: AP (CS-A) ReSch


# Get Empty Slots For A Day

In [9]:
day = input("Enter the day (Monday, Tuesday, etc.): ")
emptySlots = getEmptySlots(day)
print(f"Empty slots on {day}:")
for room, empty in emptySlots.items():
    if empty == True:
        print(f"Room: {room}, Empty: {empty}")

Enter the day (Monday, Tuesday, etc.): Tuesday
Empty slots on Tuesday:
Room: B-Digital, Empty: True


# Get Least Busy Classroom In Week

In [10]:
leastBusyClassroom = getLeastBusyClassroom()
print(f"The least busy classroom is: {leastBusyClassroom}")

The least busy classroom is: B-DLD Lab
